In [1]:
import pandas as pd
import os 
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import re
import tensorflow as tf
from tensorflow.keras import preprocessing
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import imdb

In [2]:
path = './IMDB_Dataset.csv'

if os.path.isfile(path):
    pass
else:
    print("Not found '{}'".format(path))

In [3]:
df = pd.read_csv(path)

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
print(range(len(df['review'][0])))
print(range(len(df['review'])))

range(0, 1761)
range(0, 50000)


In [6]:
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [7]:
for i in range(len(df)):
    df['review'][i] = df['review'][i].replace(['<br />',':','...'],'')

TypeError: replace() argument 1 must be str, not list

In [ ]:
df.head()

In [ ]:
df['review'][0]

# one hot encoding

In [ ]:
for i in range(len(df)):
    if df['sentiment'][i]=='positive':
        df['sentiment'][i] = 1
    else:
        df['sentiment'][i] = 0

In [ ]:
df.tail()

In [ ]:
len(df.loc[df['sentiment']==0])

In [ ]:
print(df['sentiment'].unique())

In [ ]:
labels = df['sentiment'].unique()
ratio = [len(df.loc[df['sentiment']==0]),len(df.loc[df['sentiment']==1])]

plt.pie(ratio, labels=labels, autopct='%.1f%%')
plt.show()

# train_test_split

In [ ]:
train = df[:30000]
validation = df[30000:40000]
test = df[40000:]

In [ ]:
# 각 항목의 갯수 확인
print('train : ',len(train))
print('validation : ',len(validation))
print('test : ',len(test))

## 갯수확인

In [ ]:
train.shape

In [ ]:
validation.shape

In [ ]:
test.shape

In [ ]:
x_train = train['review']
y_train = train['sentiment']

x_validation = validation['review'] 
y_validation = validation['sentiment']

x_test = test['review']
y_test = test['sentiment']

In [ ]:
x_train.shape

# tokenizer

In [ ]:
tokenizer = preprocessing.text.Tokenizer()
# fit_on_texts()는 samples 리스트에서 문장을 읽어서
# word_index를 만들어 준다
tokenizer.fit_on_texts(x_train)

# 만들어진 word_index를 확인
word_index = tokenizer.word_index

# samples의 문장을 word_index를 활용하여 문자를 숫자로 변환
sequences = tokenizer.texts_to_sequences(x_train)
# 입력된 문장과 레이블을 np.array타입으로 변환
input_sequences = np.array(sequences)
labels = np.array(train['sentiment'])

In [ ]:
print('수치화된 텍스트 데이터 : \n',sequences[0])
print('sequences type : {}'.format(type(sequences)))
print('\n단어 인덱스 :\n',word_index)
print('\n라벨 : ',labels)

In [ ]:
# train data안에 있는 리뷰 하나씩 들고 와서 s에 집어넣는다
len_result = [len(s) for s in x_train]

print('리뷰의 최대 길이 : {}'.format(np.max(len_result)))
print('리뷰의 평균 길이 : {}'.format(np.mean(len_result)))

plt.subplot(1,2,1)
plt.boxplot(len_result)
plt.subplot(1,2,2)
plt.hist(len_result, bins=50)
plt.show()

In [ ]:
unique_elements, counts_elements = np.unique(y_train, return_counts=True)
print("각 레이블에 대한 빈도수:")
print(np.asarray((unique_elements, counts_elements)))

In [ ]:
word_to_index = imdb.get_word_index()
index_to_word={}

# IMDB데이터가 특수하게 인덱스 3번부터 시작하도록 정한 규칙
# index_to_word : 앞에 세개는 '패딩','문서시작','사전에 없음'을 위한 인덱스
# index_to_word[3] : 빈도수가 가장 높은 단어
for key, value in word_to_index.items():
    index_to_word[value+3] = key

In [ ]:
print('빈도수 상위 1등 단어 : {}'.format(index_to_word[4]))

In [ ]:
print('빈도수 상위 3938등 단어 : {}'.format(index_to_word[3941]))

## GRU로 IMDB 리뷰 긍정/부정 분류하기

In [ ]:
import re
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [ ]:

DIM = 10000

# 문장을 vectorize
def vectorize_sequences(sequences, dimension=DIM):
    # 크기가 (len(sequences), dimension))이고 모든 원소가 0인 행렬을 만듭니다
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # results[i]에서 특정 인덱스의 위치를 1로 만듭니다
    return results

In [ ]:
# 훈련 데이터를 벡터로 변환합니다
# text로 된 문장 -> vector
x_train = vectorize_sequences(x_train)
# 테스트 데이터를 벡터로 변환합니다
x_test = vectorize_sequences(x_train)
# 베리데이션 데이터를 벡터로 변환합니다
x_validation = vectorize_sequences(validation)


# 레이블을 벡터로 바꿉니다
y_train = np.asarray(y_train).astype('float32')
y_test = np.asarray(y_test).astype('float32')
y_validation = np.asarray(y_validation).astype('float32')